In [1]:
import pandas as pd
import re
from os import listdir
from tqdm import tqdm
from anytree import NodeMixin, RenderTree, PreOrderIter

In [2]:
def toRow(df, entry):
    row = {}
    row['job'] = entry['job']
    row['page'] = df.loc[entry['nodes'][-1]].page.values[0]
    row['file'] = df.loc[entry['nodes'][-1]].filename.values[0]
    levels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
    for i, node in enumerate(entry['nodes']):
        row[levels[i]] = ' '.join(df.loc[node].text.values)
    row['amount'] = df.loc[entry['nodes'][-1][-2]].text
    return row


def get_patern_of_bullet(String):
    regx = [
        ('\d{5}-\d+$', 70),
        ('[1-9][0-9]*(\.[1-9][0-9]*)*\)$', 20),
        ('\(\d*(\.?\d*)*\)$', 50),
        ('[1-9][0-9]*(\.[1-9][0-9]*)+$', 2),
        ('[1-9][0-9]*\.$', 1)
    ]

    for r, l in regx:
        if re.match(r, String):
            if l in [2, 20, 50]:
                l = String.count('.') + l
            return r, l
    return '', 0

In [3]:
df = pd.read_csv('bkk-bud_cleaned_reindex.csv', index_col='index')
df = df.sort_values(by=['filename', 'page', 'line_num']).reset_index(drop=True)
fpl_group = df.groupby(['filename', 'page', 'line_num'])

In [4]:
bulletFlag = False
entry = []
entries = []

for i, group in tqdm(fpl_group):
    lineText = group.text.values
    if not i[2]:
        continue

    if (lineText[0].startswith('งาน') and i[2] <= 2) or (lineText[0].startswith('งาน:')):
        entries.append(('job', group.index.to_list()))
        continue

    if ' ' in group.iloc[0].text:
        bullet = get_patern_of_bullet(group.iloc[0].text.split(' ')[0])
    else:
        bullet = get_patern_of_bullet(group.iloc[0].text)

    if lineText[0].startswith('งาน'):
        if (lineText[-1] == 'บาท'):
            lineText = lineText[:-2]
        job = ' '.join(lineText)
        continue
    elif bullet[1]:
        bulletFlag = True

    if bulletFlag:
        entry += group.index.to_list()
        if lineText[-1] == 'บาท':
            bulletFlag = False
            entries.append(('list', entry))
            entry = []


100%|██████████| 43226/43226 [00:13<00:00, 3257.55it/s]


In [5]:
class Budget(NodeMixin):
    def __init__(self, id, name, idx, bullet, parent=None, children=None):
        super(Budget, self).__init__()
        self.name = name
        self.id = id
        self.idx = idx
        self.amount = ''
        if len(idx) > 2:
            self.amount = df.loc[idx[-2]].text
        elif len(name.split(' ')) > 2:
            self.amount = name.split(' ')[-2]

        self.bullet = bullet
        self.parent = parent
        if children:
            self.children = children
    def __repr__(self):
        return self.name


In [6]:
BUDGET_RUNNING_ID = 0
root = Budget(BUDGET_RUNNING_ID, 'BKK2022', [-1], -2)
BUDGET_RUNNING_ID += 1

job_root = None
curr = None

for l, entry in tqdm(entries):
  entryText = ' '.join(df.loc[entry].text.values)

  if l == 'job':
    job_root = Budget(BUDGET_RUNNING_ID, entryText, entry, -1, root)
    BUDGET_RUNNING_ID += 1
    curr = job_root
    continue

  entry_bullet = get_patern_of_bullet(df.loc[entry[0]].text.split(' ')[0])
  if curr == job_root:
     curr = Budget(BUDGET_RUNNING_ID, entryText, entry, entry_bullet[1], job_root)
     BUDGET_RUNNING_ID += 1
  else:
    while curr.bullet != job_root and entry_bullet[1] <= curr.bullet:
      curr = curr.parent
    curr = Budget(BUDGET_RUNNING_ID, entryText, entry, entry_bullet[1], curr)
    BUDGET_RUNNING_ID+=1
  

100%|██████████| 27749/27749 [00:16<00:00, 1661.55it/s]


In [31]:
has_parent = pd.DataFrame(
    [
        {'id': node.id, 'parent': node.parent.id}
        for node in PreOrderIter(root) if node.parent
    ])
item = pd.DataFrame(
    [
        {
            'id': node.id,
            'text': node.name, 
            'amount': node.amount, 
            'filename': df.loc[node.idx[0]].filename, 
            'page': df.loc[node.idx[0]].page,
            'line_num': df.loc[node.idx[0]].line_num
            }
        for node in PreOrderIter(root)
        if node.is_leaf
    ]
)

has_parent.to_csv('2022bkk-buget-has_parent.csv', index=0)
item.to_csv('2022bkk-buget-item.csv', index=0)

In [32]:
has_parent.groupby('parent').apply(lambda x: x.id.values)

parent
0        [1, 35, 63, 85, 109, 132, 162, 187, 212, 238, ...
1                                              [2, 23, 29]
2                                          [3, 10, 15, 19]
3                                       [4, 5, 6, 7, 8, 9]
10                                        [11, 12, 13, 14]
                               ...                        
27730                                       [27731, 27735]
27731                                [27732, 27733, 27734]
27735                                       [27736, 27737]
27738                                       [27739, 27740]
27741    [27742, 27743, 27744, 27745, 27746, 27747, 277...
Length: 8694, dtype: object

In [35]:
item[item.amount.apply(lambda x: not x.isdigit())].sample(10)

,id,text,amount,filename,page,line_num
16447,23965,01101-1 อัตราเดิม 5 อัตรา 1867 750 บาท,1867 750,65069.pdf,39,4
3839,5527,01205-1 เงินเพิ่มการครองชีพชั่วคราวของลูกจ้างป...,63865 7,65036.pdf,37,10
6290,9105,01206-1 เงินช่วยเหลือค่าครองชีพของลูกจ้างประจํ...,2 150,65042.pdf,43,13
1714,2472,01202-1 เงินเพิ่มค่าจ้างประจํา 340 800 บาท,340 800,65030.pdf,36,10
9476,13719,01206-1 เงินช่วยเหลือค่าครองชีพของลูกจ้างประจํ...,2 7120,65050.pdf,56,13
12084,17591,02101-1 ค่าจ้างชั่วคราว 3 อัตรา 312 800 บาท,312 800,65057.pdf,49,12
15620,22720,01109-1 เงินช่วยเหลือค่าครองชีพของข้าราชการ 2 ...,2 7120,65066.pdf,62,8
3844,5534,03217-1 เงินสมทบกองทุนประกันสังคม 554 112 บาท,554 112,65036.pdf,37,17
12811,18658,01101-1 อัตราเดิม 1 อัตรา 575 500 บาท,575 500,65059.pdf,47,4
5942,8595,2.2 ค่าใช้สอย ฯ192 100 บาท,ฯ192 100,65041.pdf,47,25
